<a href="https://colab.research.google.com/github/carlmaps/Visualizaion-Projects/blob/master/EarthsSurface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### EarthsSurface
This is just a simple data visulaization of the earth surface temperature using the data set form Berkely Earth Surface Temperature (BEST)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import plotly as py
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


%matplotlib inline
init_notebook_mode(connected=True)

pio.renderers.default = 'colab'

In [0]:
!pip install kaggle
# Import kaggle.json from google drive
# This snippet will output a link which needs authentication from any google account
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
    q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
# print(kaggle_api_key)
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

! mkdir ~/.kaggle
! cp /content/.kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d berkeleyearth/climate-change-earth-surface-temperature-data

!mkdir earthsurface
!unzip climate-change-earth-surface-temperature-data.zip -d earthsurface

In [0]:
df = pd.read_csv('/content/earthsurface/GlobalTemperatures.csv')
df.head()

**Date**: *starts in 1750 for average land temperature and 1850 for max and min land temperatures and global ocean and land temperatures* <br/>
**LandAverageTemperature**: *global average land temperature in celsius* <br/>
**LandAverageTemperatureUncertainty**: *the 95% confidence interval around the average* <br/>
**LandMaxTemperature**: *global average maximum land temperature in celsius* <br/>
**LandMaxTemperatureUncertainty**:* the 95% confidence interval around the maximum land temperature* <br/>
**LandMinTemperature**: *global average minimum land temperature in celsius* <br/>
**LandMinTemperatureUncertainty**: *the 95% confidence interval around the minimum land temperature* <br/>
**LandAndOceanAverageTemperature**: *global average land and ocean temperature in celsius* <br/>
**LandAndOceanAverageTemperatureUncertainty**: *the 95% confidence interval around the global average land and ocean temperature*

In [0]:
df.shape

In [0]:
df.info()

In [0]:
df.describe().transpose()

In [0]:
df.isna().sum()

In [0]:
import missingno as mns

In [0]:
mns.matrix(df, color=(0.15, 0.5, 0.2))

In [0]:
df_aveLandTemp = df.copy(deep=True)

In [0]:
df_aveLandTemp = df_aveLandTemp[['dt','LandAverageTemperature']]

In [0]:
df_aveLandTemp.isna().sum()

In [0]:
df_aveLandTemp.dropna(inplace=True)

In [0]:
df_aveLandTemp['dt'] = df_aveLandTemp['dt'].apply(lambda x:x[0:4])

In [0]:
df_aveLandTemp.head()

In [0]:
df_aveLandTemp = df_aveLandTemp.groupby(df_aveLandTemp['dt'])['LandAverageTemperature'].mean().reset_index()

In [0]:
df_aveLandTemp.dtypes

In [0]:
fig = px.line(data_frame=df_aveLandTemp, x=df_aveLandTemp.dt, 
              y=df_aveLandTemp.LandAverageTemperature,
              template='plotly_dark', title='Earths Ave Land Temperature(°C) from 1870 - 2015')

fig.show()

In [0]:
df_aveLandOceanTemp = df.copy(deep=True)

In [0]:
df_aveLandOceanTemp = df_aveLandOceanTemp[['dt','LandAndOceanAverageTemperature']]

In [0]:
df_aveLandOceanTemp.isna().sum()

In [0]:
df_aveLandOceanTemp.dropna(inplace=True)

In [0]:
df_aveLandOceanTemp['dt'] = df_aveLandOceanTemp['dt'].apply(lambda x:x[0:4])

In [0]:
df_aveLandOceanTemp.head()

In [0]:
df_aveLandOceanTemp = df_aveLandOceanTemp.groupby(['dt'])['LandAndOceanAverageTemperature'].mean().reset_index()
df_aveLandOceanTemp.head()

In [0]:
fig = px.line(data_frame=df_aveLandOceanTemp, x=df_aveLandOceanTemp.dt, 
              y=df_aveLandOceanTemp.LandAndOceanAverageTemperature,
              template='plotly_dark', title='Earths Ave Land and Ocean Temperature(°C) from 1850 - 2015')

fig.show()

In [0]:
dfbyCountry = pd.read_csv('/content/earthsurface/GlobalLandTemperaturesByCountry.csv')
dfbyCountry.head()

In [0]:
dfbyCountry.shape

In [0]:
dfbyCountry.describe().transpose()

In [0]:
dfbyCountry.dtypes

In [0]:
dfbyCountry.isna().sum()

In [0]:
dfbyCountry.Country.unique()

In [0]:
dfbyCountry.rename(columns={'United Kingdom (Europe)':'United Kingdom','France (Europe)':'France','Denmark (Europe)':'Denmark','Netherlands (Europe)':'Netherlands'}, inplace=True)

In [0]:
dfbyCountry['dt'] = dfbyCountry['dt'].apply(lambda x:x[0:4])

In [0]:
desc = dfbyCountry.groupby(['dt', 'Country'])['AverageTemperature'].mean().reset_index().sort_values(by = 'AverageTemperature' ,ascending=False)
desc.head(10)
#dfbyCountry.sort_values(by='AverageTemperature',ascending=False)[:10]

In [0]:
byCountry = dfbyCountry.groupby(['Country'])['AverageTemperature'].mean().reset_index().sort_values(by = 'AverageTemperature' ,ascending=False)
byCountry.head(10)

In [0]:
fig_byCountry = px.bar(byCountry.head(10).sort_values(by='AverageTemperature',ascending=True), x='AverageTemperature', y='Country', template="plotly_dark",
                text='AverageTemperature', orientation='h', color_discrete_sequence=px.colors.qualitative.Alphabet)

fig_byCountry.update_layout(
    title_text = 'AverageTemperature for the Top 20 Countries',
    title_x = 0.5,  
    )

In [0]:
fig = go.Figure(data=go.Choropleth(
    locations = desc['Country'],
    locationmode = 'country names',
    z = desc['AverageTemperature'],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
    
))

fig.update_layout(
    title_text = 'AverageTemperature',
    title_x = 0.5,
    template="plotly_white",
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)

fig.show()